In [11]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def get_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

def generate_signal(stock_data):
    avg_volume_15_days = stock_data['Volume'].tail(15).mean()
    today_volume = stock_data.iloc[-1]['Volume']
    if today_volume > avg_volume_15_days:
        return "Buy Signal"
    else:
        return "No Signal"

def main():
    stock_symbol = ['BSE.NS', 'HAL.NS', 'MCX.NS', 'POLYCAB.NS', 'CAMS.NS', 'CDSL.NS', 'ASIANPAINT.NS', 'PIDILITIND.NS',  'DMART.NS', 'IRCTC.NS', 'IRFC.NS' ]
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=15)).strftime('%Y-%m-%d')
    stock_data = get_stock_data(stock_symbol, start_date, end_date)
    # signal = generate_signal(stock_data)

    # print("Start "+ start_date)
    # print("End "+ end_date)
    # print(f"Stock: {stock_symbol}")
    # print(f"Today's Volume: {stock_data.iloc[-1]['Volume']}")
    # print(f"Average Volume (Last 15 days): {stock_data['Volume'].tail(15).mean()}")
    # print(f"Signal: {signal}")

if __name__ == "__main__":
    main()


[*********************100%%**********************]  11 of 11 completed


In [17]:
stock_symbol = ['BSE.NS', 'HAL.NS', 'MCX.NS', 'POLYCAB.NS', 'CAMS.NS', 'CDSL.NS', 'ASIANPAINT.NS', 'PIDILITIND.NS',  'DMART.NS', 'IRCTC.NS', 'IRFC.NS' ]
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=15)).strftime('%Y-%m-%d')
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
volume_df = stock_data["Volume"]
average_volume_df = volume_df.mean().to_frame()
average_volume_df.columns = ["15_Days_Avg_volume"]
average_volume_df


[                       0%%                      ]

[*********************100%%**********************]  11 of 11 completed


,15_Days_Avg_volume
ASIANPAINT.NS,552770.7
BSE.NS,558987.7
CAMS.NS,226399.7
CDSL.NS,607922.7
DMART.NS,534960.5
HAL.NS,1805230.5
IRCTC.NS,6976347.7
IRFC.NS,96478753.3
MCX.NS,439697.0
PIDILITIND.NS,265873.6


In [50]:
symbol_list = {
            "ETF": ['SILVERBEES.NS', 'BANKBEES.NS', 'BANKETF.NS', 'ITBEES.NS', 'MON100.NS', 'ICICIB22.NS', 'GOLDBEES.NS'],
            "Monopoly": ['BSE.NS', 'HAL.NS', 'MCX.NS', 'POLYCAB.NS', 'CAMS.NS', 'CDSL.NS', 'ASIANPAINT.NS'],
            "EV": ['BHEL.NS', 'JBMA.NS', 'OLECTRA.NS', 'M&M.NS', 'TATAMTRDVR.NS', 'DIXON.NS'],
            "IT": ['WIPRO.NS', 'HCLTECH.NS', 'LTIM.NS','TATAELXSI.NS', 'INFY.NS', 'TCS.NS'],
            "Hospitals": ['NH.NS', 'FORTIS.NS'],
            "Speciality Chemicals": ['SRF.NS', 'GALAXYSURF.NS','NAVINFLUOR.NS', 'RADICO.NS', 'FLUOROCHEM.NS', 'NAVINFLUOR.NS', 'ALKYLAMINE.NS', 'BALAMINES.NS', 'PIIND.NS' ],
            "Consumer": ['NESTLEIND.NS', 'VBL.NS', 'POLYCAB.NS', 'ITC.NS', 'TATACONSUM.NS', 'GODREJCP.NS', 'HINDUNILVR.NS', 'TITAN.NS', 'AMBER.NS'],
            "Bank": ['SBIN.NS', 'PNB.NS', 'HDFCBANK.NS'],
            "PSU" : ['GAIL.NS', 'MGL.NS', 'IGL.NS', 'HINDCOPPER.NS', 'NATIONALUM.NS', 'COALINDIA.NS'],
            "Defence": ['PARAS.NS', 'HAL.NS', 'MAZDOCK.NS']
}

def getAvgVolume(symbol_list, noofdays):
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=noofdays)).strftime('%Y-%m-%d')
    stock_data = yf.download(symbol_list, start=start_date, end=end_date)
    volume_df = stock_data["Volume"]
    average_volume_df = volume_df.mean().to_frame()
    average_volume_df.columns = [f"{noofdays}_Days_Avg_volume"]
    average_volume_df = average_volume_df.rename_axis('Symbol').reset_index()
    return average_volume_df


signals = []
for key, value in symbol_list.items():
    volume_15_day_avg_df = getAvgVolume(value, 15)
    volume_today_df = getAvgVolume(value, 1)

    volume_df = pd.merge(volume_today_df, volume_15_day_avg_df, on='Symbol')

    volume_df.loc[volume_df['1_Days_Avg_volume'] < volume_df['15_Days_Avg_volume'], 'signal'] = ''
    volume_df.loc[volume_df['1_Days_Avg_volume'] > volume_df['15_Days_Avg_volume'], 'signal'] = 'BUY'

    volume_df = volume_df[volume_df["signal"] == "BUY"]
    volume_df['PERC_CHANGE'] = ((volume_df['1_Days_Avg_volume'] - volume_df['15_Days_Avg_volume'])/volume_df['1_Days_Avg_volume'])*100
    
    signals.append(volume_df.to_dict(orient='records'))

print(len(signals))

signals = [item for sublist in signals for item in sublist]
signals_df = pd.DataFrame(signals)
signals_df.sort_values(by='PERC_CHANGE', ascending=False)

[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  6 of 6 completed
[*********************100%%**********************]  6 of 6 completed
[*********************100%%**********************]  6 of 6 completed
[*********************100%%**********************]  6 of 6 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  8 of 8 completed
[*********************100%%**********************]  8 of 8 completed
[*********************100%%**********************]  9 of 9 completed
[*********************100%%**********************]  9 of 9 completed
[*********************100%%*******

,Symbol,1_Days_Avg_volume,15_Days_Avg_volume,signal,PERC_CHANGE
10,INFY.NS,26751876.0,8270611.2,BUY,69.083995
1,ITBEES.NS,23486370.0,7270348.6,BUY,69.044392
13,TCS.NS,6811771.0,2185665.6,BUY,67.913402
21,TATACONSUM.NS,6017892.0,1941333.3,BUY,67.740642
12,TATAELXSI.NS,156832.0,65456.7,BUY,58.263173
11,LTIM.NS,899810.0,404783.3,BUY,55.014581
14,WIPRO.NS,18032219.0,8218589.2,BUY,54.422752
9,HCLTECH.NS,5861439.0,2713094.4,BUY,53.712827
19,HINDUNILVR.NS,2979505.0,1384058.6,BUY,53.547364
16,GALAXYSURF.NS,81436.0,39471.0,BUY,51.531264


In [57]:
symbol_list = {
            "ETF": ['SILVERBEES.NS', 'BANKBEES.NS', 'BANKETF.NS', 'ITBEES.NS', 'MON100.NS', 'ICICIB22.NS', 'GOLDBEES.NS'],
            "Indices": ['^NSEI', '^NSEBANK', '^BSESN', '^NSEMDCP50', '^CNXIT', '^DJI' ], # '^CNXMID', '^CNXSMALL', 'NIFTYSMLCAP250.NS'
            "Monopoly": ['BSE.NS', 'HAL.NS', 'MCX.NS', 'POLYCAB.NS', 'CAMS.NS', 'CDSL.NS', 'ASIANPAINT.NS']
}

for key, value in symbol_list.items():
    stock_data_all = yf.download(value, period='7d', interval='1d')
    close_prices = stock_data_all['Close']
    close_prices = close_prices.reset_index()
    percentage_difference = (close_prices.iloc[:, 1:].pct_change() * 100).dropna()
    percentage_difference["Date"] = close_prices["Date"]
    date_column = percentage_difference.pop('Date')
    percentage_difference.insert(0, 'Date', date_column)
    print(percentage_difference)
    

[*********************100%%**********************]  7 of 7 completed
        Date  BANKBEES.NS  BANKETF.NS  GOLDBEES.NS  ICICIB22.NS  ITBEES.NS  \
1 2024-01-05    -0.120103   -0.118041    -0.281482     0.237813   1.208455   
2 2024-01-08    -1.373686   -1.511446    -0.545730    -0.819586  -0.922660   
3 2024-01-09    -0.473230   -0.498915     0.321669     0.641510   0.356070   
4 2024-01-10     0.384122    0.382939    -0.188612    -0.983142   0.327508   
5 2024-01-11    -0.014480    0.181252    -0.151168     0.043645  -0.108806   
6 2024-01-12     0.581299    0.559612     0.283879     0.981562   4.956426   

   MON100.NS  SILVERBEES.NS  
1  -1.259774      -0.226346  
2   0.007545      -0.155963  
3   1.675852       0.639030  
4   0.608800      -0.536200  
5   0.531327      -0.085115  
6  -0.359679      -0.255573  
[*********************100%%**********************]  6 of 6 completed
        Date    ^BSESN    ^CNXIT      ^DJI  ^NSEBANK     ^NSEI  ^NSEMDCP50
1 2024-01-05  0.248551  1.2855